In [2]:
import pandas as pd

In [36]:
def get_securitie_type(name):
    name = name.lower().split(' ')
    keywords = {
        'cedear': 'cedear',
        'on': 'obligacion negociable',
        'obligacion': 'obligacion negociable',
        'bon': 'bono',
        'vto': 'bono',
        'letra': 'letra del tesoro',
        'fondo': 'fondo comun de inversion',
        'fci': 'fondo comun de inversion',
        'fondo comun': 'fondo comun de inversion'
    }

    for key, value in keywords.items():
        if key in name:
            return value
    return 'accion'

def get_financial_instrument_type(securitie_type):
    securitie_type = securitie_type.lower()

    if securitie_type in ['cedear', 'accion', 'fondo comun de inversion']:
        return 'renta variable'
    elif securitie_type in ['bono', 'obligacion negociable', 'letra del tesoro']:
        return 'renta fija'
    else:
        return 'desconocido'

# def get_securitie_dimension():


# Leer con pandas un archivo csv
df_bullma = pd.read_csv('../data/silver/bullma_portfolio_2024-09-29.csv', sep=',')

# convierto todas las columnas en minusculas
df_bullma.columns = df_bullma.columns.str.lower()




df_bullma['securitie_type'] = df_bullma['nombre'].apply(get_securitie_type)
df_bullma['financial_instrument_type'] = df_bullma['securitie_type'].apply(get_financial_instrument_type)

# cambio el nombre de la columna 'nombre' por 'full_name'
df_bullma.rename(columns={'nombre': 'full_name'}, inplace=True)

# elimino la columna 'cantidad', 'ultimo precio', 'ppc', 'total'
df_bullma.drop(columns=['cantidad', 'ultimo precio', 'ppc', 'total'], inplace=True)

# elimino la columna 'tipo' si existe
if 'tipo' in df_bullma.columns:
    df_bullma.drop(columns=['tipo'], inplace=True)

# creo una columna llamada 'dividend_yield' y 'par_value' con valores nulos
df_bullma['dividend_yield'] = None
df_bullma['par_value'] = None
df_bullma['full_name'] = df_bullma['full_name'].str.upper()


df_bullma

,ticket,full_name,securitie_type,financial_instrument_type,dividend_yield,par_value
0,AUSO,AUTOPISTA DEL SOL,accion,renta variable,None,None
1,CEPU,CENTRAL PUERTO,accion,renta variable,None,None
2,DGCU2,DISTRIB DE GAS CUYANA,accion,renta variable,None,None
3,FIPL,FIPLASTO,accion,renta variable,None,None
4,GGAL,GRUPO FINANCIERO GALICIA,accion,renta variable,None,None
5,MIRG,MIRGOR,accion,renta variable,None,None
6,PAMP,PAMPA ENERGIA,accion,renta variable,None,None
7,TECO2,TELECOM ARGENTINA,accion,renta variable,None,None
8,TRAN,TRANSENER,accion,renta variable,None,None
9,PAAS,CEDEAR PAN AMERICAN SILVER CORP,cedear,renta variable,None,None


In [37]:
# Subo el DataFrame a la tabla golden.securities de Postgres
from sqlalchemy import create_engine



engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')

# # subo el DataFrame a la tabla golden.securities
# df.to_sql('securities', engine, schema='golden', if_exists='append', index=False)



In [38]:
# convierto la columna tickets en un string largo separado por comas y a cada string le agrego comillas simples para usarlo en un select
tickets = df_bullma['ticket'].apply(lambda x: f"'{x}'").str.cat(sep=', ')

query = f"""
SELECT *
FROM golden.securities
WHERE ticket IN ({tickets})
"""

# hago una consulta a la base de datos postgres
df_postgre = pd.read_sql(query, engine)

df_postgre

,id_securitie,ticket,full_name,securitie_type,financial_instrument_type,dividend_yield,par_value


In [39]:
# me quedo con los tickets que no estan en la base de datos
df_bullma = df_bullma[~df_bullma['ticket'].isin(df_postgre['ticket'])]

df_bullma

,ticket,full_name,securitie_type,financial_instrument_type,dividend_yield,par_value
0,AUSO,AUTOPISTA DEL SOL,accion,renta variable,None,None
1,CEPU,CENTRAL PUERTO,accion,renta variable,None,None
2,DGCU2,DISTRIB DE GAS CUYANA,accion,renta variable,None,None
3,FIPL,FIPLASTO,accion,renta variable,None,None
4,GGAL,GRUPO FINANCIERO GALICIA,accion,renta variable,None,None
5,MIRG,MIRGOR,accion,renta variable,None,None
6,PAMP,PAMPA ENERGIA,accion,renta variable,None,None
7,TECO2,TELECOM ARGENTINA,accion,renta variable,None,None
8,TRAN,TRANSENER,accion,renta variable,None,None
9,PAAS,CEDEAR PAN AMERICAN SILVER CORP,cedear,renta variable,None,None
